In [1]:
%set_env SHELL=/bin/bash
%set_env OMP_NUM_THREADS=8
%set_env VECLIB_MAXIMUM_THREADS=1
%set_env ASE_CP2K_COMMAND=cp2k_shell.ssmp

env: SHELL=/bin/bash
env: OMP_NUM_THREADS=8
env: VECLIB_MAXIMUM_THREADS=1
env: ASE_CP2K_COMMAND=cp2k_shell.ssmp


In [2]:
# General
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.constants import Boltzmann, Avogadro
from copy import deepcopy

# For building things
from ase import Atom, Atoms
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.build import molecule, surface, add_adsorbate, add_vacuum, sort
from ase.visualize import view
import nglview as nv

# Unit Conversions and Fixing Atoms
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal
from ase.constraints import FixedPlane, FixedLine, FixAtoms

# ASE Calculators
#from plumed import Plumed
from ase.calculators.cp2k import CP2K
from ase.calculators.lj import LennardJones
#from ase.calculators.plumed import Plumed
from ase.calculators.idealgas import IdealGas

# Geometry Optimizations and Normal Mode Analysis
from ase.optimize import LBFGS, FIRE
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

# EOS fitting for Unit Cells
from ase.eos import EquationOfState

# Molecular Dynamics
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT

%matplotlib inline
%reload_ext autoreload
%autosave 2


Autosaving every 2 seconds


In [3]:
def myCP2KCalculator(functional, ecut):
    """Creates a CP2K calculator object with Settings for Production Runs
    tag -> label for cp2k output
    functional -> Either PBE+D3, BEEFVDW, rVV10, or LDA+FermiDirac
    ecut -> PW Kinetic Energy Cutoff (Rydberg)
    """
    
    if functional == "PBE+D3":
        xc = "PBE"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &VDW_POTENTIAL
        POTENTIAL_TYPE PAIR_POTENTIAL
        &PAIR_POTENTIAL
          R_CUTOFF 15.0
          TYPE DFTD3
          CALCULATE_C9_TERM .FALSE.
          REFERENCE_FUNCTIONAL PBE
          PARAMETER_FILE_NAME dftd3.dat
        &END PAIR_POTENTIAL
      &END VDW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
  &END DFT
&END FORCE_EVAL
'''
        
    elif functional == "PBE+D3+surface":
        xc = "PBE"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .TRUE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &VDW_POTENTIAL
        POTENTIAL_TYPE PAIR_POTENTIAL
        &PAIR_POTENTIAL
          R_CUTOFF 15.0
          TYPE DFTD3
          CALCULATE_C9_TERM .FALSE.
          REFERENCE_FUNCTIONAL PBE
          PARAMETER_FILE_NAME dftd3.dat
        &END PAIR_POTENTIAL
      &END VDW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
  &END DFT
&END FORCE_EVAL
'''
        
    elif functional == "optB88-vdw":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
     &XC
      &XC_FUNCTIONAL
        &LIBXC
          FUNCTIONAL GGA_X_OPTB88_VDW
        &END LIBXC
        &PW92
        &END PW92
      &END XC_FUNCTIONAL
      &vdW_POTENTIAL
         DISPERSION_FUNCTIONAL NON_LOCAL
         &NON_LOCAL
           TYPE DRSLL
           VERBOSE_OUTPUT
           KERNEL_FILE_NAME vdW_kernel_table.dat
         &END NON_LOCAL
      &END vdW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''

        
    elif functional == "optB88-vdw+surface":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .TRUE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
     &XC
      &XC_FUNCTIONAL
        &LIBXC
          FUNCTIONAL GGA_X_OPTB88_VDW
        &END LIBXC
        &PW92
        &END PW92
      &END XC_FUNCTIONAL
      &vdW_POTENTIAL
         DISPERSION_FUNCTIONAL NON_LOCAL
         &NON_LOCAL
           TYPE DRSLL
           VERBOSE_OUTPUT
           KERNEL_FILE_NAME vdW_kernel_table.dat
         &END NON_LOCAL
      &END vdW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "optB86B-vdw":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
     &XC
      &XC_FUNCTIONAL
        &LIBXC
          FUNCTIONAL GGA_X_OPTB86B_VDW
        &END LIBXC
        &PW92
        &END PW92
      &END XC_FUNCTIONAL
      &vdW_POTENTIAL
         DISPERSION_FUNCTIONAL NON_LOCAL
         &NON_LOCAL
           TYPE DRSLL
           VERBOSE_OUTPUT
           KERNEL_FILE_NAME vdW_kernel_table.dat
         &END NON_LOCAL
      &END vdW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''
        
    elif functional == "optPBE-vdw":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
     &XC
      &XC_FUNCTIONAL
        &LIBXC
          FUNCTIONAL GGA_X_OPTPBE_VDW
        &END LIBXC
        &PW92
        &END PW92
      &END XC_FUNCTIONAL
      &vdW_POTENTIAL
         DISPERSION_FUNCTIONAL NON_LOCAL
         &NON_LOCAL
           TYPE DRSLL
           VERBOSE_OUTPUT
           KERNEL_FILE_NAME vdW_kernel_table.dat
         &END NON_LOCAL
      &END vdW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "rVV10":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
       &XC_FUNCTIONAL
         &GGA_X_RPW86
         &END GGA_X_RPW86
         &GGA_C_PBE
         &END GGA_C_PBE
       &END XC_FUNCTIONAL
       &vdW_POTENTIAL
          DISPERSION_FUNCTIONAL NON_LOCAL
          &NON_LOCAL
            TYPE RVV10
            PARAMETERS 6.3 0.0093
            VERBOSE_OUTPUT
            KERNEL_FILE_NAME rVV10_kernel_table.dat
            CUTOFF 40
          &END NON_LOCAL
       &END vdW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
  &END DFT
&END FORCE_EVAL
'''
        
        
    elif functional == "optB86B-vdw+surface":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .TRUE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
     &XC
      &XC_FUNCTIONAL
        &LIBXC
          FUNCTIONAL GGA_X_OPTB86B_VDW
        &END LIBXC
        &PW92
        &END PW92
      &END XC_FUNCTIONAL
      &vdW_POTENTIAL
         DISPERSION_FUNCTIONAL NON_LOCAL
         &NON_LOCAL
           TYPE DRSLL
           VERBOSE_OUTPUT
           KERNEL_FILE_NAME vdW_kernel_table.dat
         &END NON_LOCAL
      &END vdW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''
        
    elif functional == "optPBE-vdw+surface":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .TRUE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
     &XC
      &XC_FUNCTIONAL
        &LIBXC
          FUNCTIONAL GGA_X_OPTPBE_VDW
        &END LIBXC
        &PW92
        &END PW92
      &END XC_FUNCTIONAL
      &vdW_POTENTIAL
         DISPERSION_FUNCTIONAL NON_LOCAL
         &NON_LOCAL
           TYPE DRSLL
           VERBOSE_OUTPUT
           KERNEL_FILE_NAME vdW_kernel_table.dat
         &END NON_LOCAL
      &END vdW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "rVV10+surface":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .TRUE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
       &XC_FUNCTIONAL
         &GGA_X_RPW86
         &END GGA_X_RPW86
         &GGA_C_PBE
         &END GGA_C_PBE
       &END XC_FUNCTIONAL
       &vdW_POTENTIAL
          DISPERSION_FUNCTIONAL NON_LOCAL
          &NON_LOCAL
            TYPE RVV10
            PARAMETERS 6.3 0.0093
            VERBOSE_OUTPUT
            KERNEL_FILE_NAME rVV10_kernel_table.dat
            CUTOFF 40
          &END NON_LOCAL
       &END vdW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "optB88-vdw+FermiDirac+MP":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &KPOINTS
        SCHEME MONKHORST-PACK 4 4 4
    &END KPOINTS
    &SCF
      ADDED_MOS 10
      &SMEAR ON
          METHOD FERMI_DIRAC
          ELECTRONIC_TEMPERATURE [K] 300
      &END SMEAR
      &MIXING .TRUE.
          METHOD BROYDEN_MIXING
      &END MIXING
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
     &XC
      &XC_FUNCTIONAL
        &LIBXC
          FUNCTIONAL GGA_X_OPTB88_VDW
        &END LIBXC
        &PW92
        &END PW92
      &END XC_FUNCTIONAL
      &vdW_POTENTIAL
         DISPERSION_FUNCTIONAL NON_LOCAL
         &NON_LOCAL
           TYPE DRSLL
           VERBOSE_OUTPUT
           KERNEL_FILE_NAME vdW_kernel_table.dat
         &END NON_LOCAL
      &END vdW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''
        
    elif functional == "optB86B-vdw+FermiDirac+MP":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &KPOINTS
        SCHEME MONKHORST-PACK 4 4 4
    &END KPOINTS
    &SCF
      ADDED_MOS 10
      &SMEAR ON
          METHOD FERMI_DIRAC
          ELECTRONIC_TEMPERATURE [K] 300
      &END SMEAR
      &MIXING .TRUE.
          METHOD BROYDEN_MIXING
      &END MIXING
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
     &XC
      &XC_FUNCTIONAL
        &LIBXC
          FUNCTIONAL GGA_X_OPTB86B_VDW
        &END LIBXC
        &PW92
        &END PW92
      &END XC_FUNCTIONAL
      &vdW_POTENTIAL
         DISPERSION_FUNCTIONAL NON_LOCAL
         &NON_LOCAL
           TYPE DRSLL
           VERBOSE_OUTPUT
           KERNEL_FILE_NAME vdW_kernel_table.dat
         &END NON_LOCAL
      &END vdW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''
        
    elif functional == "optPBE-vdw+FermiDirac+MP":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &KPOINTS
        SCHEME MONKHORST-PACK 4 4 4
    &END KPOINTS
    &SCF
      ADDED_MOS 10
      &SMEAR ON
          METHOD FERMI_DIRAC
          ELECTRONIC_TEMPERATURE [K] 300
      &END SMEAR
      &MIXING .TRUE.
          METHOD BROYDEN_MIXING
      &END MIXING
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
     &XC
      &XC_FUNCTIONAL
        &LIBXC
          FUNCTIONAL GGA_X_OPTPBE_VDW
        &END LIBXC
        &PW92
        &END PW92
      &END XC_FUNCTIONAL
      &vdW_POTENTIAL
         DISPERSION_FUNCTIONAL NON_LOCAL
         &NON_LOCAL
           TYPE DRSLL
           VERBOSE_OUTPUT
           KERNEL_FILE_NAME vdW_kernel_table.dat
         &END NON_LOCAL
      &END vdW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "rVV10+FermiDirac+MP":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &KPOINTS
        SCHEME MONKHORST-PACK 4 4 4
    &END KPOINTS
    &SCF
      ADDED_MOS 10
      &SMEAR ON
          METHOD FERMI_DIRAC
          ELECTRONIC_TEMPERATURE [K] 300
      &END SMEAR
      &MIXING .TRUE.
          METHOD BROYDEN_MIXING
      &END MIXING
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
       &XC_FUNCTIONAL
         &GGA_X_RPW86
         &END GGA_X_RPW86
         &GGA_C_PBE
         &END GGA_C_PBE
       &END XC_FUNCTIONAL
       &vdW_POTENTIAL
          DISPERSION_FUNCTIONAL NON_LOCAL
          &NON_LOCAL
            TYPE RVV10
            PARAMETERS 6.3 0.0093
            VERBOSE_OUTPUT
            KERNEL_FILE_NAME rVV10_kernel_table.dat
            CUTOFF 40
          &END NON_LOCAL
       &END vdW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
  &END DFT
&END FORCE_EVAL
'''
        
    else:
        xc = "LDA"
        inp = ''''''
        
    calc = CP2K(
        auto_write=False,
        basis_set="DZVP-MOLOPT-SR-GTH",
        basis_set_file="BASIS_MOLOPT",
        charge=0,
        cutoff = ecut*Rydberg,
        debug = False,
        force_eval_method = "Quickstep",
        xc = xc,
        inp = inp,
        max_scf = 50,
        poisson_solver ="auto",
        potential_file = "POTENTIAL",
        pseudo_potential = "GTH-PBE",
        stress_tensor = True,
        print_level = "LOW",
    )
    return calc


In [10]:
def MyEOSFitting(atoms, calc):
    
    dirname = "Simulations/"+calc.label
    
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    else:
        pass
    
    cwd = os.getcwd()
    os.chdir(dirname)
    
    atoms.calc = calc
    cell = atoms.get_cell()
    traj = Trajectory('eos.traj', 'w')
    for x in np.linspace(0.95, 1.05, 5):
        atoms.set_cell(cell * x, scale_atoms=True)
        atoms.get_potential_energy()
        traj.write(atoms)
        
    configs = read('eos.traj@0:5')  # read 5 configurations
    
    # Extract volumes and energies:
    volumes = [atoms.get_volume() for atoms in configs]
    energies = [atoms.get_potential_energy() for atoms in configs]
    eos = EquationOfState(volumes, energies)
    
    v0, e0, B = eos.fit()
    
    s = (v0/cell.volume)**(1./3.)
    atoms.set_cell(cell*s, scale_atoms=True)
    print("Scalar: ", s)
    print("Lattice Vectors: ")
    print(cell*s)
    print("Evaluating at Minimum:", atoms.get_potential_energy())
    print("Bulk Modulus [GPa]:", B / kJ * 1.0e24)
    
    os.chdir(cwd)
    
    eos.plot()

In [5]:
fal = read("Resources/xyz/furfural/FAL.xyz")
view(fal, viewer="ngl")

In [11]:
beta = read("Resources/cif/tmc/Beta-Mo2C_mp-1221498_conventional_standard.cif")
calc = myCP2KCalculator("optB88-vdw+FermiDirac+MP", 400.0)
view(beta, viewer="ngl")

In [12]:
MyEOSFitting(beta, calc)

Exception ignored in: <function Cp2kShell.__del__ at 0x7f51180b44c0>
Traceback (most recent call last):
  File "/home/woodrowwilson/Programs/ase/dev/ase/calculators/cp2k.py", line 507, in __del__
    assert rtncode == 0  # child process exited properly?
AssertionError: 


KeyboardInterrupt: 

In [53]:
def CreateProbeOnSlab(uc, probe, miller_indices, nlayers, vacuum):
    write("unit_cell.xyz", uc)
    write("probe.xyz", probe)
    
    slab = surface(read("unit_cell.xyz"), miller_indices, nlayers, vacuum)
    system = slab.repeat([3,4] + [1])
    system = sort(system, system.positions[:,2])
    
    box = system.get_cell()
    x = 0.5*(box[0][0] + box[1][0] + box[2][0])
    y = 0.5*(box[0][1] + box[1][1] + box[2][1])
    
    add_adsorbate(system, read("probe.xyz"), 2.5, position=(x, y), offset=None, mol_index=0)
    
    c = FixAtoms(indices=[atom.index for atom in system if atom.position[2] < 0.5*np.sum(system.get_cell()[:,2])])
    system.set_constraint(c)
    
    return system

In [54]:
system = CreateProbeOnSlab(beta, fal, [1,0,1], 4, 7.5)

In [58]:
view(system, viewer="ngl")

In [60]:
system.calc = myCP2KCalculator("optB88-vdw+surface", 400.0)

In [65]:

cwd = os.getcwd()
os.chdir(cwd)

dirname = system.calc.label
if not os.path.isdir(dirname):
    os.mkdir("Simulations/"+dirname)
else:
    pass

system.get_potential_energy()  
os.chdir(cwd)



KeyboardInterrupt: 